In [19]:
#Import dependencies
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import matplotlib 
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression,Lasso
from sklearn.model_selection import ShuffleSplit,cross_val_score,GridSearchCV
from sklearn.tree import DecisionTreeRegressor
import catboost as cb
import lightgbm as lgb
from sklearn.feature_selection import VarianceThreshold,chi2
import pickle
import json

In [20]:
#Import dataset
housepricedata = pd.read_csv('Bengaluru_House_Data.csv')
housepricedf = housepricedata.copy()

In [21]:
housepricedata.head()

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00


## Data Preprocessing and Feature Engineering

In [11]:
def isfloat(x):
    '''convert totsl_sqrt to numerical values'''
    try:
        float(x)
    except:
        return False
    return True
def convert_sqft_to_num(x):
    '''To convert totsl_sqrt to numerical values'''
    tokens = x.split('-')
    if len(tokens) == 2:
        return (float(tokens[0])+float(tokens[1]))/2
    try:
        return float(x)
    except:
        return None   
def iqr(dff):
    ''' Return iqr range'''
    q1 = dff.quantile(0.25)
    q3 = dff.quantile(0.75)
    iqr = q3-q1
    return q1-1.5*iqr,q3+1.5*iqr

def remove_pps_outliers(df):
    ''' Remove the outlies in price_per_sqrft'''
    df_out = pd.DataFrame()
    for key, subdf in df.groupby('location'):
        q1,q3 = iqr(df['price_per_sqrft'])
        reduced_df = subdf[(subdf.price_per_sqrft>(q1)) & (subdf.price_per_sqrft<=(q3))]
        df_out = pd.concat([df_out,reduced_df],ignore_index=True)
    return df_out

def remove_pps_outliers_(df):
    ''' Remove the outlies in price_per_sqrft'''
    df_out = pd.DataFrame()
    for key, subdf in df.groupby('location'):
        m = np.mean(subdf.price_per_sqrft)
        st = np.std(subdf.price_per_sqrft)
        reduced_df = subdf[(subdf.price_per_sqrft>(m-st)) & (subdf.price_per_sqrft<=(m+st))]
        df_out = pd.concat([df_out,reduced_df],ignore_index=True)
    return df_out

### Data Preprocessing 

In [12]:
#drop null values from the data
housepricedf.dropna(inplace=True)

#convert size to bhk as numerical values
housepricedf['bhk'] = housepricedf['size'].apply(lambda x:int(x.split(' ')[0]))

#convert total_sqrt to numerical
housepricedf['total_sqft'] = housepricedf['total_sqft'].apply(convert_sqft_to_num)

#let's creat new features 
#price per sqrft
housepricedf['price_per_sqrft'] = housepricedf['price']*100000/housepricedf['total_sqft']

#remove the locations with less obervations
housepricedf['location'] = housepricedf['location'].apply(lambda x : x.strip())
location_stats = housepricedf['location'].value_counts(ascending=False)
location_stats_less_than_10 = location_stats[location_stats<=10]
housepricedf['location'] = housepricedf['location'].apply(lambda x: 'other' if x in location_stats_less_than_10 else x)

#remove obervations with sqrft/bhk less than 300(Since minimum sqrft is 300 for single room)
housepricedf = housepricedf[~((housepricedf['total_sqft']/housepricedf['bhk'])<300)]

### Outliers

In [ ]:
#remove outlies in price_per_sqrft using iqr range
housepricedf = remove_pps_outliers_(housepricedf)

#remove obeservations having more number of bathrooms than bhk
housepricedf = housepricedf[housepricedf['bath']<housepricedf['bhk']+2]

#drop columns - size
housepricedf.drop(columns=['size','price_per_sqrft'],inplace=True)

#Get copy of housedf
housedfcopy = housepricedf.copy()

### Encoding

In [ ]:
#Encoding location features - 
locationdummydf = pd.get_dummies(housepricedf['location'])
locationdummydf.drop(columns = ['other'],inplace=True)
housepricedf = pd.concat([housepricedf,locationdummydf],axis=1)
housepricedf.drop(columns=['location'],inplace=True)

### Feature Selection

In [13]:
#drop columns which are not relevent for price predictions
housepricedf.drop(columns=['area_type','availability','society','balcony'],inplace=True)

##Variance Threshold Feature selection for continuos features
# var_thresh = VarianceThreshold(threshold=0)
# var_thresh.fit(housepricedf)
# var_thresh.get_support()

## Chi2 test for categorical features
#catfeatures = ['area_type','size']
## split the data into train and test for feature selection
# pvalues = chi2(x_train,y_train)

## Model Training

In [5]:
#split the data int train and test
Y = housepricedf['price']
X = housepricedf.drop(['price'],axis=1)
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.2,shuffle=True)

### 1 - Linear Regression

In [6]:
lr_clf = LinearRegression()
#Fit the data using Linear Regression model
lr_clf.fit(x_train,y_train)

#Cross validation scores 
cv = ShuffleSplit(n_splits=5,test_size=0.2,random_state=2021)
cross_valscore = cross_val_score(LinearRegression(),X,Y,cv=cv)
print('Cross Val Score is ',cross_valscore)

Cross Val Score is  [0.81928163 0.78652451 0.79986841 0.77168779 0.8490252 ]


### Gridsearch CV
Gridsearch for three models : Linear Regression, Decision Tree Regression,Lasso Regression

In [7]:
def best_model_gridsearch(x,y):
    algos = {
            'lr':{
                 'model':LinearRegression(),
                 'params':{ 
                          'normalize' :[True,False]}
                 },
            'lasso':{
                    'model' : Lasso(),
                    'params' : {
                                'alpha' : [1,2],
                                'selection' : ['random','cyclic']
                                }
                    },
            'dt' :{
                  'model' : DecisionTreeRegressor(),
                  'params':{
                            'criterion':['mse','friedman_mse'],
                            'splitter' : ['best','random']
                           }
                  }
            }
    scores = []
    cv = ShuffleSplit(n_splits=5,test_size=0.2,random_state=2021)
    for algo_name,config in algos.items():
        gs = GridSearchCV(config['model'],config['params'],cv=cv,return_train_score=False)
        gs.fit(X,Y)
        scores.append({
                    'model':algo_name,
                    'best_score':gs.best_score_,
                    'best_params':gs.best_params_
                     })
    return pd.DataFrame(scores,columns=['model','best_score','best_params'])

In [8]:
best_model_gridsearch(X,Y)

,model,best_score,best_params
0,lr,0.805278,{'normalize': False}
1,lasso,0.682503,"{'alpha': 1, 'selection': 'random'}"
2,dt,0.718483,"{'criterion': 'friedman_mse', 'splitter': 'best'}"


In [9]:
#Test for some obeservations
def predict_price(location,sqft,bath,bhk):    
    loc_index = np.where(X.columns==location)[0][0]

    x = np.zeros(len(X.columns))
    x[0] = sqft
    x[1] = bath
    x[2] = bhk
    if loc_index >= 0:
        x[loc_index] = 1

    return lr_clf.predict([x])[0]

### Save Model

In [10]:
#save model using Pickle
with open('banglore_home_prices_model.pickle','wb') as f:
    pickle.dump(lr_clf,f)

In [11]:
#save location names for deploymnet 
columns = {
    'data_columns' : [col.lower() for col in X.columns]
}
with open("columns.json","w") as f:
    f.write(json.dumps(columns))

## Advanced Models
Train advanced models like XGBoost and Catboost Regressor

## Catboost Regressor

In [12]:
# Split the train data into train and validation
train, validation = train_test_split(housedfcopy, test_size=0.2)

In [13]:
avoid_cols = ['price']
feats = [col for col in train.columns if col not in avoid_cols]
target = 'price'

# Categorical features
cat_cols = ['location']
# Categorical features index 
cat_indx = [i for i,c in enumerate(feats) if c in cat_cols]

# Catboost pool
val_pool = cb.Pool(validation[feats].values, validation[target], cat_features=cat_indx)

# Model setup
cat_reg = cb.CatBoostRegressor(iterations=3000,
                                   learning_rate=0.005,
                                   l2_leaf_reg=5,
                                   eval_metric='RMSE',
                                   random_seed = 42,
                                   metric_period=500,
                                   od_wait=20,
                                   use_best_model=True
                                   )
# Model fit
cat_reg.fit(X=train[feats], y=train[target], eval_set=val_pool, 
            cat_features=cat_indx, use_best_model=True, verbose=True, plot=False)

0:	learn: 82.2919071	test: 77.9147188	best: 77.9147188 (0)	total: 169ms	remaining: 8m 27s
500:	learn: 47.3421621	test: 41.8013412	best: 41.8013412 (500)	total: 14.9s	remaining: 1m 14s
Stopped by overfitting detector  (20 iterations wait)

bestTest = 40.53590959
bestIteration = 909

Shrink model to first 910 iterations.


## LightGBM Regressor

In [14]:
# Spli the data into train and test
Y = housepricedf['price']
X = housepricedf.drop(['price'],axis=1)
train_x,validation_x,train_y ,validation_y = train_test_split(X,Y,test_size=0.2)

In [15]:
def light_gbm_model_run(train_x, train_y, validation_x, validation_y):
    params = {
        "objective" : "regression",
        "metric" : "rmse",
        "num_leaves" :70,
        "min_data_in_leaf" :20,
        "learning_rate" : 0.001,
        "bagging_seed" : 42,
        "verbosity" : -1,
        "seed": 42}
    
    # Given its a regression case, I am using the RMSE as the metric.

    lg_train = lgb.Dataset(train_x, label=train_y)
    lg_validation = lgb.Dataset(validation_x, label=validation_y)
    evals_result_lgbm = {}

    model_light_gbm = lgb.train(params, lg_train, 5000,
                      valid_sets=[lg_train, lg_validation],
                      early_stopping_rounds=100,
                      verbose_eval=500,
                      evals_result=evals_result_lgbm )

    return model_light_gbm
#Run Lightgbm code
lgbmmodel = light_gbm_model_run(train_x, train_y, validation_x, validation_y)

C:\Users\Asus\anaconda3\envs\py37\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\Asus\anaconda3\envs\py37\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\Asus\anaconda3\envs\py37\lib\site-packages\lightgbm\engine.py:260: UserWarning: 'evals_result' argument is deprecated and will be removed in a future release of LightGBM. Pass 'record_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'eva

Training until validation scores don't improve for 100 rounds
[500]	training's rmse: 63.0838	valid_1's rmse: 56.8678
[1000]	training's rmse: 53.9382	valid_1's rmse: 46.6169
[1500]	training's rmse: 49.8339	valid_1's rmse: 42.436
[2000]	training's rmse: 47.7255	valid_1's rmse: 40.7353
[2500]	training's rmse: 46.3517	valid_1's rmse: 39.732
[3000]	training's rmse: 45.361	valid_1's rmse: 39.1191
[3500]	training's rmse: 44.7313	valid_1's rmse: 38.8866
[4000]	training's rmse: 44.3492	valid_1's rmse: 38.8335
Early stopping, best iteration is:
[4380]	training's rmse: 44.1289	valid_1's rmse: 38.8135
